# Build a Dashboard Application with Plotly Dash

In this lab, you will be building a Plotly Dash application for users to perform interactive visual analytics on SpaceX launch data in real-time.

This dashboard application contains input components such as a dropdown list and a range slider to interact with a pie chart and a scatter point chart. You will be guided to build this dashboard application via the following tasks:

- TASK 1: Add a Launch Site Drop-down Input Component
- TASK 2: Add a callback function to render success-pie-chart based on selected site dropdown
- TASK 3: Add a Range Slider to Select Payload
- TASK 4: Add a callback function to render the success-payload-scatter-chart scatter plot

Note:Please take screenshots of the Dashboard and save them. Further upload your notebook to github.

The github url and the screenshots are later required in the presentation slides.

The completed dashboard application should look sthg like this
![Alt text](image.png)

After visual analysis using the dashboard, you should be able to obtain some insights to answer the following five questions:

- Which site has the largest successful launches?
- Which site has the highest launch success rate?
- Which payload range(s) has the highest launch success rate?
- Which payload range(s) has the lowest launch success rate?
- Which F9 Booster version (v1.0, v1.1, FT, B4, B5, etc.) has the highest launch success rate?

# IMPORT Libraries and Data

In [69]:
import pandas as pd
import wget
import numpy as np

In [70]:
import wget
a = wget.download('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv')
df = pd.read_csv(a)

In [71]:
# this is a skeleton dashboard application
b = wget.download('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/spacex_dash_app.py')


## TASK 1: Add a Launch Site Drop-down Input Component

We have **four different launch sites** and we would like to first see which one has the largest success count.

Then, we would like to select one specific site and check its detailed success rate (class=0 vs. class=1).

As such, we will need a dropdown menu to let us select different launch sites.

Find and complete a commented dcc.Dropdown(id='site-dropdown',...) input with following attributes:
- **id** attribute with value site-dropdown
- **options** attribute is a list of dict-like option objects (with label and value attributes). You can set the **label** and **value** all to be the launch site names in the **spacex_df** and you need to include the default **All** option. 
     -  Example: options=[{'label': 'All Sites', 'value': 'ALL'},{'label': 'site1', 'value': 'site1'}, ...]

- **value** attribute with default dropdown value to be ALL meaning all sites are selected
- **placeholder** attribute to show a text description about this input area, such as **Select a Launch Site here**
- **searchable** attribute to be True so we can enter keywords to search launch sites

In [72]:
import pandas as pd
import dash # main library used to create web-based applications
# Import the Dash class from the dash library
from dash import Dash, html, dash_table
# import html (short for dash_html_components), this is used for building the layout of components. We use 'html' as acronym
# Import dash_core_components (acronym 'dcc') also used for building components such as dropdown, sliders, checkboxes, buttons, geolocation, graphs ...
from dash import dcc
# import input, output from dash.dependencies so we use them without referring to dash.dependencies
from dash.dependencies import Input
from dash.dependencies import Output
import plotly.express as px # plotly express for creating the figures. dash library is built on top of plotly

In [73]:
# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("spacex_launch_dash.csv")
print(spacex_df.shape)
print(spacex_df.head(5))

(56, 7)
   Unnamed: 0  Flight Number  Launch Site  class  Payload Mass (kg)  \
0           0              1  CCAFS LC-40      0                0.0   
1           1              2  CCAFS LC-40      0                0.0   
2           2              3  CCAFS LC-40      0              525.0   
3           3              4  CCAFS LC-40      0              500.0   
4           4              5  CCAFS LC-40      0              677.0   

  Booster Version Booster Version Category  
0  F9 v1.0  B0003                     v1.0  
1  F9 v1.0  B0004                     v1.0  
2  F9 v1.0  B0005                     v1.0  
3  F9 v1.0  B0006                     v1.0  
4  F9 v1.0  B0007                     v1.0  


In [74]:
print('Launch Site Names', spacex_df['Launch Site'].unique())
print('Number of Launch Sites', len(spacex_df['Launch Site'].unique())) # so there are 4 launch sites
max_payload = spacex_df['Payload Mass (kg)'].max()
print(max_payload) # 9600
min_payload = spacex_df['Payload Mass (kg)'].min()
print(min_payload) # 0
# Since min is 0 and max is 9,600 so range of payload is 0:10,000

Launch Site Names ['CCAFS LC-40' 'VAFB SLC-4E' 'KSC LC-39A' 'CCAFS SLC-40']
Number of Launch Sites 4
9600.0
0.0


In [75]:
# Initialize the dash app
app = dash.Dash("__Lab_2-Dashboard__")

In [76]:
# 1st Set up the app layout using the 2 libraries we imported html and dcc
# we will create 4 components in this dash app
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard', # html is the html_dash_components library we imported.
                                        # 'SpaceX Launch Records Dashboard' is the dashboard title
                                        style={'textAlign': 'center', 'color': '#503D36','font-size': 40}),
                                # Component/Task 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                html.Div(children=[
                                    html.Label('Launch Site'),
                                    # dcc is the dash_core_components library we imported for dropdown
                                    dcc.Dropdown(id = 'site-dropdown', # add an id for each dash component to reference it
                                                 options = [{'label':'All Sites', 'value':'ALL'},
                                                            {'label':'CCAFS LC-40', 'value':'CCAFS LC-40'}, 
                                                            {'label':'VAFB SLC-4E', 'value':'VAFB SLC-4E'},
                                                            {'label':'KSC LC-39A', 'value':'KSC LC-39A'},
                                                            {'label':'CCAF SLC-40', 'value':'CCAFS LC-40'}],
                                                 multi = False,
                                                 placeholder = 'Select a Launch Site',
                                                 searchable = True) # make it searchable so we can enter keywords in the search box
                                    ]),
                                    html.Br(), # create a line break
                                # Component/Task 2: Add a pie chart to show the total successful launches count for all sites
                                html.Div(children=[
                                    html.Label('Sucess Pie Chart'),
                                    dcc.Graph(id = 'pie-chart',
                                              figure = px.pie(spacex_df['Launch Site'], # we can plot the figure using plotly.express (acronym px)
                                                              names = 'Category',
                                                              title = 'Breakdown of Sites for all Launches')
                                              )
                                ]),
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                                                
                                html.Br(), # create a lin break

                                # Component/Task 3: Add a slider to select payload range
                                html.Div(children=[
                                    html.Label('Payload range (Kg)'),
                                    dcc.RangeSlider(id='payload-slider',
                                                    min=min(spacex_df['Payload Mass (kg)']),
                                                    max=max(spacex_df['Payload Mass (kg)']),
                                                    # Make the steps every 1000
                                                    step=1000,
                                                    # make tik marks
                                                    marks={i: str(i) for i in range(min(spacex_df['Payload Mass (kg)']), max(spacex_df['Payload Mass (kg)']), 1000)},
                                                    # make the value between the min and max
                                                    value=[min(spacex_df['Payload Mass (kg)']), max(spacex_df['Payload Mass (kg)'])]
                                                    )]
                                ),

                                # Component/Task 4: Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
                                html.Div(children=[
                                    html.Label('Success Rate as Function of Payload Mass (kg'),
                                    dcc.Graph(id='scatter-chart')
                                    ])
                                

# Callback functions make the dashboard interactive
# Callback functions are python functions that get automatically called by dash. The function runs and updates its output
# Add Callback for Component/Task 2: Add a callback function for `site-dropdown` as input, `pie-chart` as output
# There are 2 sections of a callback (1) decorator (for input and output) and (2) the function
# Section 1 - Decorator
@app.callback(
    Output(component_id='pie-chart', component_property='figure'),
    [Input(component_id='site-dropdown', component_property='value')]
    #Output(component_id = 'pie-chart', component_property = 'figure'),
    #Input(component_id = 'site-dropdown', component_property = 'value')
)
# Section 2 - Function
# this function makes the pie chart dependent on the dropdown
def update_pie_chart(selected_site):
    # 'selected_site' is a parameter in the function 'update_pie_chart' here
    # Filter the dataframe based on the selected launch site
    # this is if user chose to see success rates of all sites in 1 pie chart
    if selected_site == 'ALL':
        filtered_df = spacex_df.copy() # then don't filter, use all the dataframe
        title = 'Total Successful Launches for All Sites'
    else:
        filtered_df = spacex_df[spacex_df['Launch Site'] == selected_site] # this is if user chose only the rows for the filtered data when its the site you want 
        title = f'Total Successful Launches for {selected_site}'
    
    # Create the pie chart for success rate (called 'class')
    pie_chart = px.pie(filtered_df,
                       names='class', # specify class as input for pie chart (either 0,1)
                       title=title
                       )
    return pie_chart

# Add Callback for Component/Task 4: Add a scatter chart to show the correlation between payload and launch success        
# Section 1: Decorator
@app.callback(
    Output(component_id = 'scatter-chart', component_property = 'figure'),
    [Input(component_id = 'site-dropdown', component_property = 'value'),
     Input(component_id = 'payload-slider', component_property='value')]
)
# Section 2: Function
# this function makes the scatter plotdependent on the dropdown
def update_scatter_chart(selected_site, payload_range):
    # Filter the dataframe based on the selected launch site and payload range
    if selected_site == 'ALL':
        filtered_df = spacex_df.copy()
    else:
        filtered_df = spacex_df[spacex_df['Launch Site'] == selected_site]
    
    filtered_df = filtered_df[
        # payload_range is not defined earlier, its a parameter that Dash will populate with the selected range when the callback is triggered. 
        (filtered_df['Payload Mass (kg)'] >= payload_range[0]) &
        (filtered_df['Payload Mass (kg)'] <= payload_range[1])
    # if the user selects a payload range from 1000 kg to 5000 kg, then payload_range[0] would be 1000, and payload_range[1] would be 5000.
    ]
    
    # Create the scatter chart
    scatter_chart = px.scatter(
        filtered_df,
        x='Payload Mass (kg)',
        y='Class',
        title=f'Payload vs. Launch Success for {selected_site}',
        color='Booster Version Category'
    ) 
    return scatter_chart
    
# Run the app. By default, it runs on your local machine
# After running successfully, the created dash web app through a url default address to access the dashboard
if __name__ == '__main__':
    app.run_server()

SyntaxError: unexpected EOF while parsing (2095910533.py, line 73)

In [ ]:
def update_pie_chart(selected_site):
    # 'selected_site' is a parameter in the function 'update_pie_chart' here
    # Filter the dataframe based on the selected launch site
    # this is if user chose to see success rates of all sites in 1 pie chart
    if selected_site == 'ALL':
        filtered_df = spacex_df.copy() # then don't filter, use all the dataframe
        title = 'Total Successful Launches for All Sites'
    else:
        filtered_df = spacex_df[spacex_df['Launch Site'] == selected_site] # this is if user chose only the rows for the filtered data when its the site you want 
        title = f'Total Successful Launches for {selected_site}'
    
    # Create the pie chart for success rate (called 'class')
    pie_chart = px.pie(filtered_df,
                       names='class', # specify class as input for pie chart (either 0,1)
                       title=title
                       )
    return pie_chart


In [ ]:
update_pie_chart('CCAFS LC-40')

# TASK 2: Add a callback function to render success-pie-chart based on selected site dropdown

# TASK 3: Add a Range Slider to Select Payload

# TASK 4: Add a callback function to render the success-payload-scatter-chart scatter plot